In [1]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')

sys.path.append('../src/')

# DiffMoog Structure

DiffMoog is a customizable, modular synthesizer, with a virtually limitless number of possible structures. It is designed as 3-channels, 7-layers matrix, each cell containing a specific module or operation (i.e. oscillator, FM, filter, ADSR etc.). The sound flows through these modules in a directional manner, each cell getting audio / control input from earlier cells in the graph:

<br>
<img alt="Concept%20Map-2.jpg" height="500" src="assets/Concept Map.jpg" width="1300"/>

## Defining a Preset

Our way of controlling the synthesizer structure is through the chains mechanism. A chain is a list of Python dictionaries, each one defining a cell in the synth matrix. In this cell definition we specify which module is used in the cell, as well as the cell inputs and outputs. Let's look at a very basic, 1-channel preset:

In [2]:
OSC_ADSR = [
    {'index': (0, 0), 'operation': 'osc', 'default_connection': True},     # Our first cell is a simple oscillator
    {'index': (0, 1), 'operation': 'env_adsr', 'default_connection': True},   # audio signal from the oscillator goes into an ADSR module
]

For convenience, we defined a *default_connection* parameter which simply means that the cell takes audio input from the cell immediately before it, and outputs to the cell immediately after it.

Let's look at a more complex, multichannel preset, that also contains control signals:

In [3]:
BASIC_FLOW = [
    {'index': (0, 0), 'operation': 'lfo'},          # An LFO, that generates control signal for an FM oscillator
    {'index': (0, 1), 'operation': 'fm_sine', 'control_input': [(0, 0)], 'outputs': [(0, 2)]},    # The FM oscillator gets a control signal from the LFO
    {'index': (1, 0), 'operation': 'lfo'},
    {'index': (1, 1), 'operation': 'fm_square', 'control_input': [(1, 0)], 'outputs': [(0, 2)]},
    {'index': (2, 0), 'operation': 'lfo'},
    {'index': (2, 1), 'operation': 'fm_saw', 'control_input': [(2, 0)], 'outputs': [(0, 2)]},
    {'index': (0, 2), 'operation': 'mix', 'audio_input': [(0, 1), (1, 1), (2, 1)], 'outputs': [(0, 3)]},  # Signals from the 3 channels are mixed and passed on
    {'index': (0, 3), 'operation': 'env_adsr', 'default_connection': True},
    {'index': (0, 4), 'operation': 'lowpass_filter', 'default_connection': True}
]

Another capability of DiffMoog is **online modularity** - we can define multiple outputs for a cell, and switch between them randomly during sound generation. For example in this preset:

In [4]:
MODULAR = [
    {'index': (0, 0), 'operation': 'lfo_sine', 'outputs': [(0, 6), (1, 1)], 'switch_outputs': True},   # This LFO generates control signal that can go into an FM module or into the tremolo module at the end
    {'index': (1, 1), 'operation': 'fm', 'control_input': [(0, 0)], 'outputs': [(0, 2), (1, 2), (2, 2)],
     'switch_outputs': True, 'allow_multiple': False, 'active_prob': 0.75},
    {'index': (0, 2), 'operation': 'fm_sine', 'control_input': [(1, 1)], 'outputs': [(0, 3)]},
    {'index': (1, 2), 'operation': 'fm_saw', 'control_input': [(1, 1)], 'outputs': [(0, 3)]},
    {'index': (2, 2), 'operation': 'fm_square', 'control_input': [(1, 1)], 'outputs': [(0, 3)]},
    {'index': (0, 3), 'operation': 'mix', 'audio_input': [(0, 2), (1, 2), (2, 2)], 'default_connection': True},
    {'index': (0, 4), 'operation': 'env_adsr', 'default_connection': True},
    {'index': (0, 5), 'operation': 'lowpass_filter', 'default_connection': True},
    {'index': (0, 6), 'operation': 'tremolo', 'control_input': [(0, 0)], 'default_connection': True}
]

To see which other operations (modules) available, check out *src/synth/synth_modules.py*
To see other suggested presets see *src/synth/synth_chains.py*

# Create Sounds using DiffMoog

Once we have defined the structure of our synthesizer, we are ready to start generating sounds. Our code is designed to be used in large batches - i.e. to automatically sample random parameter combinations and generate sounds using our defined synth. Take a look at the notebook *create_dataset.py* for more about that.

To help better understand different presets and their behaviour, we provide an exploration tool that enables to generate and visualize specific sounds from a specific preset:

## Define a Preset

First, define a preset for the synthesizer structure you want to use. You can use one of the existing ones, or define your own.

In [5]:
import torchaudio

from src.synth.synth_architecture import SynthModular

from src.synth.synth_constants import SynthConstants
synth_structure = SynthConstants()

spectrogram_op = torchaudio.transforms.Spectrogram(n_fft=256)
db_op = torchaudio.transforms.AmplitudeToDB()

device = 'cuda:0'
synth_chain = 'BASIC_FLOW'   # Define your own here

synth = SynthModular(chain_name=synth_chain, synth_constants=synth_structure, device=device)

C:\Users\noamk\PycharmProjects\diffmoog\venv\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


AssertionError: Torch not compiled with CUDA enabled

## Choose Parameters

Next we need to choose the parameters for each module. We have designed a simple UI to allow you to do it here:

In [10]:
from IPython.display import display, Audio
from utils.notebook_utils import *

full_layout, output, button, output_window = create_synth_params_layout(synth_chain)


def collect_params(**d):
    parsed_synth_params = convert_flat_nb_dict_to_synth(d)
    synth.update_cells_from_dict(parsed_synth_params)
    

def generate_sound(b):
    synth.generate_signal(2.0)
    signal = synth.get_final_signal().cpu().detach()
    
    spec = db_op(spectrogram_op(signal)).numpy().squeeze()

    output_window.clear_output()
    with output_window:
        audio = Audio(signal.numpy().squeeze(), rate=16000, autoplay=False)
        display(audio)

        fig, ax = plt.subplots(1, 1, figsize=(12, 3))
        ax.plot(signal.numpy().squeeze())

        fig2, ax2 = plt.subplots(1, 1, figsize=(10, 3))
        ax2.imshow(spec, origin='lower')
        plt.show()


out = widgets.interactive_output(collect_params, output)
button.on_click(generate_sound)
display(full_layout, out)

TypeError: string indices must be integers